In [10]:
import os
import sys
import numpy as np
import pickle
from scipy.io.matlab import loadmat
from seq_brescal import PFBayesianRescal
from seq_bcomp_rescal import PFBayesianCompRescal
from seq_logit_brescal import PFBayesianLogitRescal
from scipy.spatial.distance import cosine

In [11]:
T = np.zeros([2,5,5])
T[0,0,2]=1
T[0,1,3]=1
T[1,2,4]=1
T[1,3,4]=1

n_dim=5
n_particle=1
var_x = 0.1
var_comp=1
E = np.identity(5)
max_iter=0
obs_mask = T.copy()
obs_mask = np.ones_like(T)

In [6]:
a = list()
b = list()
c = list()

models = ['brescal', 'bcomp_add', 'bcomp_mul']
models = ['brescal']
for model in models:
    if model == 'brescal':
        _model = PFBayesianRescal(n_dim, var_x=var_x, n_particles=n_particle, compute_score=False, parallel=False,
                                  sample_all=True)
        seq = _model.fit(T, obs_mask=obs_mask.copy(), max_iter=max_iter)
        R = _model.R[0]
        _model = PFBayesianRescal(n_dim, var_x=var_x, n_particles=n_particle, compute_score=False, parallel=False,
                                  sample_all=True)            
        seq = _model.fit(T, obs_mask=obs_mask.copy(), max_iter=max_iter, givenR = R)
#            a.append(np.dot(_model.E[0][0], _model.E[0][1]))
        a.append(cosine(_model.E[0][0], _model.E[0][1]))
    elif model == 'bcomp_add':
        _model = PFBayesianCompRescal(n_dim, compositionality='additive', var_x=var_x, var_comp=var_comp,
                                      n_particles=n_particle, compute_score=False)            
        seq = _model.fit(T, obs_mask=obs_mask.copy(), max_iter=max_iter, givenR=R.copy())
#             b.append(np.dot(_model.E[0][0], _model.E[0][1]))
        b.append(cosine(_model.E[0][0], _model.E[0][1]))            
    elif model == 'bcomp_mul':
        _model = PFBayesianCompRescal(n_dim, compositionality='multiplicative', var_x=var_x, var_comp=var_comp,
                                      n_particles=n_particle, compute_score=False)            
        seq = _model.fit(T, obs_mask=obs_mask.copy(), max_iter=max_iter, givenR=R.copy())
#             c.append(np.dot(_model.E[0][0], _model.E[0][1]))
        c.append(cosine(_model.E[0][0], _model.E[0][1]))            
print(np.mean(a), np.std(a))
print(np.mean(b), np.std(b))
print(np.mean(c), np.std(c))

0.687087721753 0.0
nan nan
nan nan


/Users/arongdari/.python3/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/arongdari/.python3/lib/python3.4/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/arongdari/.python3/lib/python3.4/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/Users/arongdari/.python3/lib/python3.4/site-packages/numpy/core/_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/arongdari/.python3/lib/python3.4/site-packages/numpy/core/_methods.py:116: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
_model.E[0]

array([[-0.40896548,  0.19702305,  0.94169044, -0.54650052,  0.46177452],
       [ 0.36603789, -0.45616926,  0.92168406, -0.03925254, -0.40669579],
       [ 0.15098962, -0.12414308,  0.30450357,  0.11368613,  0.08066247],
       [-0.18090938,  0.05599377,  0.31771374, -0.07433239, -0.07379061],
       [-0.76129659,  1.1224935 ,  0.22369968, -0.54502888,  0.67338503]])

In [14]:
E = _model.E[0]
for i in range(5):
    for j in range(5):
        a = 1 - cosine(E[i], E[j])
        print(i,j,a)

0 0 1.0
0 1 0.312912278247
0 2 0.357596852474
0 3 0.80241255235
0 4 0.656827876795
1 0 0.312912278247
1 1 1.0
1 2 0.78609699552
1 3 0.522247802248
1 4 -0.441958340228
2 0 0.357596852474
2 1 0.78609699552
2 2 1.0
2 3 0.322517085896
2 4 -0.30763103656
3 0 0.80241255235
3 1 0.522247802248
3 2 0.322517085896
3 3 1.0
3 4 0.420217889617
4 0 0.656827876795
4 1 -0.441958340228
4 2 -0.30763103656
4 3 0.420217889617
4 4 1.0


In [15]:
T[0]

array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [16]:
T[1]

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.]])